This is a models for a kaggle competition: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
df = (pd.read_csv('../datasets/linear_reg/kaggle_prices_train.csv')).drop(columns=['Id'])
y_train=df['SalePrice']
x_train=df.drop(columns=['SalePrice'])
x_test=(pd.read_csv('../datasets/linear_reg/kaggle_prices_test.csv')).drop(columns=['Id'])
y_test=(pd.read_csv('../datasets/linear_reg/sample_submission.csv')).drop(columns=['Id'])
id_df=pd.read_csv('../datasets/linear_reg/sample_submission.csv',usecols=['Id'])
cat_cols = ['MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood','Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType','GarageFinish', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition','LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond','BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu','GarageQual', 'GarageCond', 'PoolQC']
num_cols=[col for col in x_train.columns if col not in cat_cols]
params = {'iterations':[50,75,100,125,150],
        'learning_rate':[0.05,0.01],
        'depth':[3,5,7,9,12]}
x_train_full = x_train.copy()
x_test_full = x_test.copy()
x_train_full[cat_cols] = x_train_full[cat_cols].fillna('missing')
x_train_full[num_cols] = x_train_full[num_cols].fillna(x_train_full[num_cols].mean())
x_test_full[cat_cols] = x_test_full[cat_cols].fillna('missing')
x_test_full[num_cols] = x_test_full[num_cols].fillna(x_train_full[num_cols].mean())
grid = GridSearchCV(CatBoostRegressor(cat_features=cat_cols,loss_function='RMSE',verbose=0),params,cv=5)
grid.fit(x_train_full,y_train)
model = grid.best_estimator_
predictions = model.predict(x_test_full)
submission = pd.DataFrame({
    'Id':id_df['Id'],
    'SalePrice': predictions})
submission.to_csv('catboost_submission.csv', index=False, encoding='utf-8')
mse=mean_squared_error(y_test,predictions)
r2=r2_score(y_test,predictions)
print(f'MSE: {mse:.2f}')
print(f'R2: {r2:.2f}')

MSE: 4928614938.15
R2: -17.08
